<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import requests
from bs4 import BeautifulSoup

In [1]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 5432

In [7]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=5432
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [6]:
# текст запроса
query_3_1 = f'''
SELECT 
   COUNT(*) vacancies_cnt
FROM vacancies
'''


In [7]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,vacancies_cnt
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [8]:
# текст запроса
query_3_2 = f'''
    SELECT
        COUNT(*) employers_cnt
    FROM employers
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,employers_cnt
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [10]:
# текст запроса
query_3_3 = f'''
    SELECT
        COUNT(*) areas_cnt
    FROM areas
'''

In [12]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,areas_cnt
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [13]:
# текст запроса
query_3_4 = f'''
    SELECT
        COUNT(*) industries_cnt
    FROM industries
'''    

In [14]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,industries_cnt
0,294


***

Выводы по предварительному анализу данных.

 Данные о вакансиях - 49197. Данные о работодателях  - 23501. Данные о регионах  - 1362. Данные об отраслях / сферах деятельности - 294. 

В среднем* работодатель предлагает 2 вакансии, при этом  в регионе предлагается 36 вакансий от 17 работодателей (вероятно, преимущественно небольшие компании, поэтому у многих работодателей может не быть определенных вакансий, либо большие компании не нуждаются в наборе сотрудников). Каждая отрасль/сфера деятельности представлена 80 компаниями (вероятно, достаточно для выбора соискателем).

*с учетом особенности каждого региона, работодателя и отрасли/сферы деятельности







# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [15]:
# текст запроса
query_4_1 = f'''
    SELECT 
        a.name area, 
        COUNT(*) vacancies_cnt
    FROM areas a
    JOIN  vacancies v ON a.id = v.area_id
    GROUP BY area
    ORDER BY vacancies_cnt DESC
    LIMIT 5
'''

In [16]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area,vacancies_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [17]:
# текст запроса
query_4_2 = f'''
    SELECT 
        COUNT(*) not_empty_salary_cnt
    FROM vacancies
    WHERE salary_to is not NULL 
        or salary_from is not NULL 
'''

In [18]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,not_empty_salary_cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [19]:
# текст запроса
query_4_3 = f'''
    SELECT 
        ROUND(AVG(salary_from), 0) avg_salary_from,
        ROUND(AVG(salary_to), 0) avg_salary_to
    FROM vacancies
'''

In [20]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [21]:
# текст запроса
query_4_4 = f'''
    SELECT 
        schedule, 
        employment,
        COUNT(*) vacancies_cnt
    FROM vacancies
    GROUP BY schedule, employment
    ORDER BY vacancies_cnt DESC
'''

In [22]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
# текст запроса
query_4_5 = f'''
    SELECT 
        experience, 
        COUNT(*) vacancies_cnt
    FROM vacancies
    GROUP BY experience
    ORDER BY vacancies_cnt
'''

In [24]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий.

Количество вакансий находится в прямой зависимости от размера региона. Большинство вакансий локализованы в Москве и Санкт-Петербурге.

У примерно 50% вакансий не указана одна из границ заработной платы (24073 вакансии), вероятно, вследствие фиксированной ставки, либо обсуждении финансовых условий "по договоренности"

Средняя нижняя граница зарплатной вилки примерно 70 тысяч рублей. Средняя верхняя граница зарплатной вилки примерно 110 тысяч рублей (разница примерно в полтора раза), при этом информативность средних показателей весьма низкая с учетом разброса цифр заработных плат в регионах и больших городах.

Самым востребованным (70% вакансий) традиционно является полный день (35367 вакансий). Второе место удаленной работы (7802 вакансий) связано, вероятно, с  пандемией, высокой стоимостью жизни в крупных городах, пр.

Количество вакансий находится в обратной зависимости от увеличения опыта работы. Чаще всего ищут сотрудников с опытом работы от 1 до 3 лет (26152 вакансий). Это, вероятно, связано с тем, что данные специалисты имеют достаточный уровень опыта для выполнения рутинной работы при низкой оплате труда. Опытные специалисты, преимущественно, занимают руководящие должности при высокой оплате труда; наем, возможно, осуществляется по рекомендациям.   

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [8]:
# текст запроса
query_5_1 = f'''

    SELECT 
        e.name company_name, 
        COUNT (v.id) vacancies
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY company_name
    ORDER BY vacancies DESC
    LIMIT 5

'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,company_name,vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
# текст запроса
query_5_2 = f'''
    SELECT 
        a.name, 
        COUNT(e.id) employers_cnt, 
        COUNT(v.id) vacancies_cnt
    FROM areas a 
    LEFT JOIN vacancies v ON a.id = v.area_id
    JOIN employers e ON a.id = e.area
    WHERE v.id IS NULL
    GROUP BY a.name
    ORDER BY vacancies_cnt, employers_cnt DESC
    LIMIT 10
'''

In [28]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,name,employers_cnt,vacancies_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
5,Беларусь,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
# текст запроса
query_5_3 = f'''
    SELECT
        e.name employer_name, 
        COUNT(DISTINCT area_id) area_cnt
    FROM employers e
    JOIN vacancies v ON e.id = v.employer_id
    GROUP BY employer_name
    ORDER BY area_cnt DESC
    LIMIT 10
'''

In [30]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,employer_name,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
# текст запроса
query_5_4  = f'''
    SELECT 
        COUNT(*) employers_without_industries_cnt
    FROM employers e
    LEFT JOIN employers_industries ei ON e.id = ei.employer_id
    WHERE ei.employer_id IS NULL
'''

In [32]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,employers_without_industries_cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
# текст запроса
query_5_5 = f'''
    SELECT 
        e.name company_name
    FROM employers e
    LEFT JOIN employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.name
    HAVING COUNT(industry_id) = 4
    ORDER BY e.name
    OFFSET 2
    LIMIT 1
'''


In [34]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,company_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [35]:
# текст запроса
query_5_6 = f'''
    SELECT 
        COUNT(DISTINCT ei.employer_id) software_development_cnt
    FROM employers_industries ei
    JOIN industries i ON ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
'''

In [36]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,software_development_cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [43]:
# код для получения списка городов-миллионников
url ='https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' 
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
table = page.find('table', class_='standard sortable')
rows = table.find_all('tr')

million_city_list = []
for index,row in enumerate(rows):
    if index == 0:
        continue
    td= row.find_all('td')
    million_city_list.append(td[1].find('a').get('title'))
    
print('Список городов-миллионников (всего-{}):'.format(len(million_city_list)))
print(*million_city_list, sep='\n')    

Список городов-миллионников (всего-16):
Москва
Санкт-Петербург
Новосибирск
Екатеринбург
Казань
Нижний Новгород
Красноярск
Челябинск
Самара
Уфа
Ростов-на-Дону
Краснодар
Омск
Воронеж
Пермь
Волгоград


In [44]:
# текст запроса
city_list = ','.join(map(lambda x: "'"+x+"'",million_city_list))

query_5_7 = f'''
(
SELECT 
    a.name city_name, 
    COUNT(*) cnt
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id
WHERE a.name IN ({city_list}) AND e.name = 'Яндекс'
GROUP BY a.name
)
UNION
(
SELECT 
    'Total' city_name, 
    COUNT(*) cnt
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id
WHERE a.name IN ({city_list}) AND e.name = 'Яндекс'
)
ORDER BY cnt, city_name
'''

In [45]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,city_name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Казань,25
5,Пермь,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

Выводы по анализу работодателей:

В топе работодателей по числу вакансий находятся, преимущественно, крупные компании (IT компании, банки, связь, сырье), что, вероятно, связано не только с размером, но и внутренними особенностями управления (текучка кадров, привлечение новых сотрудников в связи с развитием бизнеса и пр)

Больше всех вакансии предлагает компания Яндекс (1933 вакансии) и при этом имеет представительство в большинстве регионов (181 регион), включая каждый из городов-миллионников  (485 вакансий), что, в свою очередь, говорит о крупном характере компании, для привлечения сотрудников которой свойственно иметь представительство во множестве локаций.

Примерно у каждой шестой компании в качестве сферы деятельности указано "Разработка программного обеспечения"

Более чем у трети работодателей не указана сфера деятельности (8419 из 23501 работодателей),  но есть компании, связанные не менее чем с четырьмя сферами деятельности (например, 2ГИС).





# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [46]:
# текст запроса
query_6_1 = f'''
    SELECT
        COUNT(*) data_cnt
    FROM vacancies
    WHERE Lower(name) LIKE '%data%' 
        OR Lower(name) LIKE '%данн%'
'''

In [47]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,data_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [48]:
# текст запроса
query_6_2 = f'''
SELECT 
    COUNT(*) junior_cnt
FROM vacancies
WHERE (lower(name) LIKE '%data scientist%' 
    OR lower(name) LIKE '%data science%' 
    OR lower(name) LIKE '%исследователь данных%' 
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
    OR lower(name) LIKE '%machine learning%'
    OR lower(name) LIKE '%машинн%обучен%') AND (lower(name) LIKE '%junior%' 
    OR lower(experience) LIKE 'нет опыта' 
    OR lower(employment) LIKE 'стажировка') 
    '''

In [49]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,junior_cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [50]:
# текст запроса
query_6_3 = f'''
SELECT
    COUNT(*) ds_vacancies_cnt
FROM vacancies 
WHERE (
    lower(name) LIKE '%data scientist%' 
    OR lower(name) LIKE '%data science%' 
    OR lower(name) LIKE '%исследователь данных%' 
    OR (name LIKE '%ML%' AND lower(name) NOT LIKE '%html%')
    OR lower(name) LIKE '%machine learning%'
    OR lower(name) LIKE '%машинн%обучен%' 
    )
    AND (
    vacancies.key_skills LIKE '%SQL%'
    OR lower(vacancies.key_skills) LIKE '%postgres%'
	  )
'''

In [51]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,ds_vacancies_cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [52]:
# текст запроса
query_6_4 = f'''
SELECT
    COUNT(*) ds_vacancies_python_cnt
FROM vacancies 
WHERE (
    lower(name) LIKE '%data scientist%' 
    OR lower(name) LIKE '%data science%' 
    OR lower(name) LIKE '%исследователь данных%' 
    OR (name LIKE '%ML%' AND lower(name) NOT LIKE '%html%')
    OR lower(name) LIKE '%machine learning%'
    OR lower(name) LIKE '%машинн%обучен%' 
    )
    AND vacancies.key_skills LIKE '%Python%'
	  
'''

In [53]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,ds_vacancies_python_cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [54]:
# текст запроса
query_6_5 = f'''
    SELECT 
        ROUND(avg(1 + length(key_skills) - length(replace(key_skills, chr(9), ''))), 2)
        ds_vacancies_key_skills_cnt
    FROM vacancies
    WHERE lower(name) LIKE '%data scientist%' 
        OR lower(name) LIKE '%data science%' 
        OR lower(name) LIKE '%исследователь данных%' 
        OR (name LIKE '%ML%' AND lower(name) NOT LIKE '%html%')
        OR lower(name) LIKE '%machine learning%'
        OR lower(name) LIKE '%машинн%обучен%' 
        
        
'''

In [55]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,ds_vacancies_key_skills_cnt
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [56]:
# текст запроса
query_6_6 = f'''
    SELECT 
        experience,
        ROUND(Avg((Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2))
        ds_avg_salary
    FROM vacancies
    WHERE (salary_to IS NOT NULL OR salary_from IS NOT NULL)
        AND 
        (
        lower(name) LIKE '%data scientist%' 
        OR lower(name) LIKE '%data science%' 
        OR lower(name) LIKE '%исследователь данных%' 
        OR (name LIKE '%ML%' AND lower(name) NOT LIKE '%html%')
        OR lower(name) LIKE '%machine learning%'
        OR lower(name) LIKE '%машинн%обучен%'
        ) 
    GROUP BY experience
'''

In [57]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,ds_avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

Выводы по предметному анализу:

1771 вакансия, связанная с данными - 3.6% от общего числа вакансий.

51 вакансия для начинающего дата-саентиста  - 2.9% от вакансий, связанных с данными и
0.1% от от общего числа вакансий, что весьма немного. Вероятно, работодатели предпочитают дата-саентистов, имеющих опыт работы. Возможно, для увеличения вакансий, критерии для начинающего дата-саентиста при приеме на работу будут в дальнейшем меняться.

201 вакансия для дата-саентистов со знанием SQL или Postgres, 351 вакансия  -  со знаничем Python, следовательно, знание Python, является более востребованным.


В целом стандартное среднее количество ключевых навыков в вакансиях для дата-саентиста  -  6.41.


Максимальный средний уровень заработной платы (243115 рублей) для дата-саентиста с опытом от 3 до 6 лет, далее с опытом от 1 года до 3 лет (139675 рублей), наименьший  - без опыта работы (74643 рублей). Для специалистов с опытом работы более 6 лет отсутствует указание размера заработной платы, вероятно, наем высококвалифицированных специалистов (возможно, на руководящую должность) осуществляется "по договоренности".











  

# Общий вывод по проекту

В рассматриваемых базах информация о 49197 вакансиях и 23501 компании (в среднем работодатель предлагает 2 вакансии). Большинство вакансий локализованы в Москве и Санкт-Петербурге, как в наиболее экономически активных регионах.

У примерно 50% вакансий не указана одна из границ заработной платы. Средняя нижняя граница зарплатной вилки примерно 70 тысяч рублей. Средняя верхняя граница зарплатной вилки примерно 110 тысяч рублей (разница примерно в полтора раза), при этом информативность средних показателей весьма низкая.

Самым востребованным (70% вакансий) традиционно является полный день. Второе место удаленной работы связано, вероятно, с  пандемией. Также можно отметить преимущество удаленной работы на фоне высокой стоимости жизни в крупных городах. Прогностически, вероятно, увеличение доли удаленной работы на фоне общей цифровизации бизнеса.

Количество вакансий находится в обратной зависимости от увеличения опыта работы. Чаще всего ищут сотрудников с опытом работы от 1 до 3 лет. Наем опытных специалистов, возможно, осуществляется по рекомендациям. 

В топе работодателей по числу вакансий находятся, преимущественно, крупные компании (IT компании, банки, связь, сырье), что, вероятно, связано не только с размером, но и внутренними особенностями управления.

Больше всех вакансии предлагает компания Яндекс которая имеет представительство в большинстве регионов, включая каждый из городов-миллионников, что, говорит о крупном характере компании, для привлечения сотрудников которой свойственно иметь представительство во множестве локаций.

Примерно у каждой шестой компании в качестве сферы дяетельности указано "Разработка программного обеспечения"
Более чем у трети работодателей не указана сфера деятельности.

3.6% от общего числа вакансий связаны с данными, что составляет определенную нишу на рынке труда, требующую более детальной оценки. Для начинающего дата-саентиста доступно 2.9% от вакансий, связанных с данными и 11% от DS-вакансий. Возможно, для увеличения количества вакансий, критерии для начинающего дата-саентиста при приеме на работу будут в дальнейшем меняться.

 В 42% (201 вакансия) в DS в качестве ключевого навыка указан SQL или Postgres. В 73% в качестве ключевого навыка указан Python (351 вакансии). Знание Python при устройстве на работу является наиболее востребованным.

Максимальный средний уровень заработной платы для дата-саентиста с опытом от 3 до 6 лет, наименьший  - без опыта работы. Для специалистов с опытом работы более 6 лет отсутствует указание размера заработной платы, вероятно, наем высококвалифицированных специалистов осуществляется по "договоренности".






Дополнительные запросы и выводы:

1.  Общее количество вакансий (ТОП-20)

In [58]:
query_7_1 = f'''
    SELECT name vacancy_name, 
    COUNT(*) vacancy_cnt
    FROM vacancies
    GROUP BY vacancy_name
    ORDER BY vacancy_cnt DESC
    LIMIT 20
'''

In [59]:
df = pd.read_sql_query(query_7_1, connection)
df

,vacancy_name,vacancy_cnt
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


Среди всех вакансий  наиболее популярные: 1. Системный администратор 2. Программист 1С.

2. Общее количество вакансий для дата-саентиста 

In [61]:
query_7_2 = f'''
SELECT
    COUNT(*) ds_vacancies_total_cnt
FROM vacancies 
WHERE lower(name) LIKE '%data scientist%' 
    OR lower(name) LIKE '%data science%' 
    OR lower(name) LIKE '%исследователь данных%' 
    OR (name LIKE '%ML%' AND lower(name) NOT LIKE '%html%')
    OR lower(name) LIKE '%machine learning%'
    OR lower(name) LIKE '%машинн%обучен%' 
	  
'''

In [62]:
df = pd.read_sql_query(query_7_2, connection)
df

,ds_vacancies_total_cnt
0,480


3. ТОП-20 работодателей с наибольшим количеством вакансий для дата-саентистов.

In [63]:
query_7_3 =  f'''
SELECT
e.name company_name,
COUNT(*) cnt
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE lower(v.name) LIKE '%data scientist%' 
    OR lower(v.name) LIKE '%data science%' 
    OR lower(v.name) LIKE '%исследователь данных%' 
    OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%' 
        
GROUP BY e.name
ORDER BY cnt DESC
LIMIT 20
'''

In [64]:
df = pd.read_sql_query(query_7_3, connection)
df

,company_name,cnt
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,МТС,6


На первом месте СБЕР (финансовая и технологическая компания), на втором месте Bell Integrator (системный интегратор, предоставляющий технологические услуги и инновационные сервисы) при этом Яндекс (компания в отрасли информационных технологий), имеющий наибольшее количество вакансий в целом, находится на 6 месте.

4. Распределение средней зарплаты для дата-саентистов по регионам 

In [65]:
query_7_4 = f''' 
SELECT 
    a.name area_name,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(v.name) vacancies
FROM vacancies v
JOIN areas a ON v.area_id = a.id
WHERE lower(v.name) LIKE '%data scientist%' 
    OR lower(v.name) LIKE '%data science%' 
    OR lower(v.name) LIKE '%исследователь данных%' 
    OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%' 
        
GROUP BY area_name
HAVING COUNT(v.name) > 5
ORDER BY avg_salary desc
'''

In [66]:
df = pd.read_sql_query(query_7_4, connection)
df

,area_name,avg_salary,vacancies
0,Нижний Новгород,NaN,20
1,Томск,NaN,8
2,Нур-Султан,NaN,6
3,Краснодар,NaN,6
4,Минск,NaN,9
5,Армения,268863.0,6
6,Москва,207630.0,217
7,Санкт-Петербург,173335.0,64
8,Новосибирск,166333.0,23
9,Екатеринбург,124000.0,7


 Наибольшее количество вакансий в Москве и Санкт-Петербурге при высокой средней заработной плате (2 и 3 место). Наибольшая средняя заработная плата в Армении, но количество вакансий ограничено (6).



В заключение, с точки зрения соответствия данных целям проекта создания модели машинного обучения, можно отметить проблемные места: небольшой объем данных касательно Data Science (3.6%), недостаточно информации об границах средней заработной платы и заработной платы дата-саентистов с опытом более 6 лет, сфере деятельности работодателей, низкая информативность средних показателей заработанной платы по регионам. 